# Modulos

In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import preprocessing
import matplotlib.pyplot as plt
import os
# import the relevant modules

from sklearn.cluster import KMeans
from Bio import SeqIO
import copy
import math
import glob
import re

In [ ]:
#notebook_path = os.path.abspath("./Python/variations.xls")


# Tratamento de Dados

Criar data frame e colocar primeira coluna como sample

In [ ]:
raw_data = pd.read_excel(
    "../Python/variations.xls", sheet_name="variations")
    #"./projects/covid/variations.xls", sheet_name="variations")

raw_df = pd.DataFrame(raw_data)
raw_df.rename(columns={'Unnamed: 0': 'sample'}, inplace=True)
raw_df

preencher os valores NA da primeira coluna (sample) com os valores anteriores




In [ ]:
raw_df['sample'].ffill(inplace=True)
raw_df
# test_df.tail(50)

Retirar valores NA das restantes colunas (Ref, Alt e #)

In [ ]:
raw_df = raw_df.dropna(axis=0, how="any")
raw_df

#Como alternativa:
#data_no_na = test_df[['REF', 'ALT', '#']].apply(lambda x: pd.Series(x.dropna().values)) -> alternativa para remover os NAs


Usar `pivot` para colocar a coluna sample como index, as colunas REF e ALT passam para as primeiras 2 rows 

In [ ]:
# final_data = raw_df.pivot(index="sample",
#                               columns=["REF", "ALT"], values=("#"))
# final_data.columns.name

final_df = (raw_df.set_index(['sample', 'REF', 'ALT'])
                ['#']
                .unstack(['REF', 'ALT'], fill_value=0)
                )
final_df

In [ ]:
for column in final_df.columns[1:]:
	if sum(final_df[column]) < 10:
		final_df.drop([column], axis=1, inplace=True)

final_df

Guardar em excel



In [ ]:
final_df.to_excel('final_df_new.xlsx')

# PCA analysis


* Centrar e fazer scaling aos dados
NOTA: Se as nossas amostras estivessem nas colunas, colocar `preprocessing.scale(final_data_2.T)` para fazer a transposiçao

In [ ]:
scaled_data = preprocessing.scale(final_df) # Fazer scaling dos dados para que fique centrado (0,0) 
# Como alternativa podemos usar o seguinte código para centrar os dados:
# StandardScaler().fit_transform(final_data_2)

pca = PCA(n_components=2) #n_componentes=X onde X é o numero de PC que quermos colocar no spree plot
pca.fit(scaled_data)  # calcular loading scores e variaçao para cada PCA
pca_data = pca.transform(scaled_data)
pca_data

### Scree plot para ver quantas componentes devem estar presentes no plot final. Este plot é usado para determinar o numero de fatores/principal components para uma analise de PCA.

* Calcular % de variabilidade que cada PC tem

In [ ]:
per_var = np.round(pca.explained_variance_ratio_*100, decimals=1)
per_var


* Criar labels para cada PC (PC1, PC2, PC3...) tendo em conta o tamanho da variabilidade

In [ ]:
labels = ['PC' + str(x) for x in range(1, len(per_var)+1)]
labels


* Fazer plot do spree plot

In [ ]:
plt.bar(x=range(1, len(per_var)+1), height=per_var, tick_label=labels)
plt.ylabel('Percentage of explained variability')
plt.xlabel('Principal component')
plt.title('Scree plot')
plt.show()  # grande parte da variabilidade está presente nos primeiros 9 componentes (cut-off point de 2). Estes 9 PC podem fazer uma boa representaçao geral dos dados

5. Colocar as principais coordenadas (9) numa data frame onde os rows são os samples e as colunas tem a PC label


In [ ]:

#Guardar as labels das amostras numa variavel unica de modo a colocar como indice na data frame dos PCs
sample_labels = final_df.index

pca_df = pd.DataFrame(pca_data, index=[sample_labels], columns=labels)
pca_df

In [ ]:


plt.scatter(pca_df.PC1, pca_df.PC2, alpha=0.2)
plt.title("PCA graph")
plt.xlabel('PC1- {0}%'.format(per_var[0]))
plt.ylabel('PC2- {0}%'.format(per_var[1]))
plt.show()


* KMeans clustering para identificar cluster (extrair os nossos clusters)


In [ ]:
kmeans = KMeans(n_clusters=2, random_state=0) #como estamos a olhar para 2 PC, o n_clusters vai ser 2. O metodo 

#Compute cluster centers and predict cluster indices
X_clustered = kmeans.fit_predict(pca_df)
X_clustered

In [ ]:
color_map = {0 : 'blue',
                   1 : 'red'} #alterar consoante o numero de componentes a analisar

label_color = [color_map[i] for i in X_clustered]
plt.figure(figsize = (10,10))
plt.scatter(pca_df.PC1,pca_df.PC2, c= label_color, alpha=0.3)
plt.show()


# Separar 20k sequencias em 5 ficheiros com ~ 4k


In [ ]:
ids = pd.read_excel(
    "../Python/samples_ID.xlsx")


In [ ]:
#Read fasta files with seqIO

for i in SeqIO.parse("sequences.fasta", "fasta"):
	#print(i.id)
	#print(str(i.seq)[11288:11296]) #deleçoes
	print(str(i.seq)) #deleçoes
	#print(len(i))
	break

records = list(SeqIO.parse("sequences.fasta", "fasta"))
len(records)

In [ ]:

#Estes valores de x e y tem que estar fora do for loop caso contrario o x iria fazer reset para 0 a cada iteraçao e o y tambem fazia reset para 4030 a cada iteraçao
x = 0; #começar no 1º elemento da lista de records
y = 4030; #ir até ao 4000º elemento da lista de records

for i in range(1,6):
	SeqIO.write(records[x:y], "sequence_{id}.fasta".format(id=i), "fasta")
	x = y; #atualizar o valor de X com o ultimo valor de Y usado
	y += 4000; #adicionar 4000 ao y


#Tambem podiamos fazer de forma manual:
# SeqIO.write(records[0:4001], "sequences_01.fasta", "fasta")
# SeqIO.write(records[4001:8001], "sequences_02.fasta", "fasta")
# SeqIO.write(records[8001:12001], "sequences_03.fasta", "fasta")
# SeqIO.write(records[12001:16001], "sequences_04.fasta", "fasta")
# SeqIO.write(records[16001:], "sequences_05.fasta", "fasta")


* Concatenar ficheiros csv numa data frame



In [ ]:
path = r'C:\Users\Rafael\Desktop\main\University\BioinformaticaClinica\1Semestre\FEM\Projeto\FEM\Python'
all_files = glob.glob(path + "/*.csv")
df_from_each_file = (pd.read_csv(f,sep=";") for f in all_files)
concatenated_df = pd.concat(df_from_each_file, ignore_index = True)

* Dataframe com dados que interessam (seqName, clade, substitutions, deletions insertions)

In [39]:

final_df=pd.read_csv("data_final.csv")
#final_df = concatenated_df.iloc[:,[0,1,13,14,15]] #selecionar colunas que interessam

final_df.to_csv(r'./data_final.csv', index= False, header=True)

pd.set_option('display.max_rows', 200) 


# Contar substituições, inserções e deleções

## Substituições


In [40]:
# Criar nova data frame para colocar linhas como seqName / Clade e colunas como C-T, etc

pca_df = final_df.iloc[:,[0,1]]
pca_df.head(10)
#A>T

,seqName,clade
0,Wuhan/Hu-1/2019,19A
1,Portugal/CV62/2020,20B
2,Portugal/CV63/2020,20A
3,Portugal/PT0001b/2020,20B
4,Portugal/PT0003/2020,20A
5,Portugal/PT0004/2020,20A
6,Portugal/PT0005/2020,20A
7,Portugal/PT0006a/2020,20A
8,Portugal/PT0006b/2020,20A
9,Portugal/PT0007/2020,20B


In [41]:
final_df.head(20)

,seqName,clade,substitutions,deletions,insertions
0,Wuhan/Hu-1/2019,19A,NaN,NaN,NaN
1,Portugal/CV62/2020,20B,"C241T,C3037T,C14408T,A23403G,C27046T,G28881A,G...",NaN,NaN
2,Portugal/CV63/2020,20A,"C241T,C3037T,C14408T,A23403G,C29144T",NaN,NaN
3,Portugal/PT0001b/2020,20B,"C241T,C3037T,C14408T,A23403G,C27046T,G28881A,G...",NaN,NaN
4,Portugal/PT0003/2020,20A,"C241T,C3037T,C14408T,A23403G,C29144T",NaN,NaN
5,Portugal/PT0004/2020,20A,"C241T,C3037T,C14408T,A23403G,C29144T",NaN,NaN
6,Portugal/PT0005/2020,20A,"C241T,C335T,C3037T,C14408T,G20398A,A23403G,A26...",NaN,NaN
7,Portugal/PT0006a/2020,20A,"C241T,C3037T,C14408T,A23403G,C29144T",NaN,NaN
8,Portugal/PT0006b/2020,20A,"C241T,C3037T,C14408T,A23403G,C28310T,C29144T,T...",NaN,NaN
9,Portugal/PT0007/2020,20B,"C241T,C3037T,C3373A,C14408T,A23403G,G28881A,G2...",NaN,NaN


In [44]:

for line in range(1,len(final_df)): #len(final_df)
	#print("Nova linha ########")

	substitutions = final_df.iloc[line,2] #substitutions vai dando reset a medida que vao atribuindo novos valores. /TODO: posso tentar passar isto para lista e fazer o loop por elemento de lista em vez de usar split(",")
	snv = [] #Criar nova lista por cada linha lida 

	#print(substitutions)
	#print(line)

	for content in substitutions.split(","):
		reference = content[0] #Obter o 1º character (a referencia)
		substitution = content[-1] #Obter o ultimo caracter (a substituiçao)
		output = "{0}>{1}".format(reference,substitution)
		pca_df
		snv.append(output)
		#print(snv)

	d = dict() #criar dicionario para fazer as contagens
	for key in snv:
		d[key] = d.get(key,0) + 1
	#print(d)

	for key,value in d.items():
		#print(key)
		#print(value)
		#if key not in pca_df.columns[:]:
		#	pca_df.from_dict(d)
		#print(line)
		#print(key)
		#print(value)
		pca_df.loc[line, key] = value #aqui usamos loc pois loc = label-based, ou seja, temos que especificar o nome das rows e colunas que queremos filtrar. iloc (i) - integer index-based, ou seja, temos que especificar as rows e colunas pelo index



C:\Users\Rafael\anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\Rafael\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [46]:
pca_df.head(35)


,seqName,clade,C>T,A>G,G>A,G>C,T>C,C>A,G>T,A>T,T>A,T>G,A>C,C>G
0,Wuhan/Hu-1/2019,19A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Portugal/CV62/2020,20B,4.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Portugal/CV63/2020,20A,4.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Portugal/PT0001b/2020,20B,4.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Portugal/PT0003/2020,20A,4.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Portugal/PT0004/2020,20A,4.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Portugal/PT0005/2020,20A,4.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Portugal/PT0006a/2020,20A,4.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Portugal/PT0006b/2020,20A,5.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Portugal/PT0007/2020,20B,3.0,1.0,2.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

# snv = []


# for line in final_df:
# 	for content in line.split(","): #Por cada linha na final_df na coluna substitutions, usando como separador ",":
# 		print(reference)
		


# 		#Criar sequencia de 2 até ..
# 		for i in range(2,20111):
# 			if snv not in pca_df.columns[:]:
# 				pca_df.insert(i, snv, 1)
# 		# if snv not in pca_df.iloc[0,:]:
# 		# 		append 
# 		# else:
# 		# 		count snv[i] += 1		


# pca_df

# for line in final_df:
# 	for content in final_df['substitutions']:
	#colocar o 1º carater na 1º linha
	#Colocar o ultimo carater na 2º linha
	#Se o 1º e ultimo caracter forem iguais, count += 1

## Deleções

In [47]:
#Em relaçao à ref, deleçao

record_dict = SeqIO.to_dict(SeqIO.parse("sequences.fasta", "fasta"))
print(str(record_dict["Wuhan/Hu-1/2019"].seq)[1604:1606])

#TCTGGTTTT

AT


* Para a deleçao temos que:
	* Fazer loop por cada linha da coluna 'deletions'
		* Se a linha for NaN, passar a frente
		* Caso contrario, fazer split da linha a partir do "," e guardar os resultados numa lista:
			* Ver a possibilidade de usar enumerate para guardar o indice das posiçoes (pos1 e pos2) das deleçoes, de modo a colocar tudo no mesmo sitio no final
			* Para cada lista, fazer split com "-" de modo a obter 2 posiçoes (onde ocorre a deleçao)

			* Tentar colocar um if statement que diz:
				* Se estas posiçoes contem ",", entao fazer split das virgulas. Podemos depois colocar cada um dos valores de forma individual numa lista 

	* Fazer um novo loop 
		* Passar no dicionario da sequencia fasta original de modo a ver onde houve deleçao, usando posiçao 1 e 2 calculada no loop inicial
		* 

In [ ]:
print(final_df['deletions'].head(20))

In [5]:
record_dict_2 = SeqIO.to_dict(SeqIO.parse("sequences.fasta", "fasta"))



In [36]:
final_df.head(20)

,seqName,clade,substitutions,deletions,insertions
0,Wuhan/Hu-1/2019,19A,NaN,NaN,NaN
1,Portugal/CV62/2020,20B,"C241T,C3037T,C14408T,A23403G,C27046T,G28881A,G...",NaN,NaN
2,Portugal/CV63/2020,20A,"C241T,C3037T,C14408T,A23403G,C29144T",NaN,NaN
3,Portugal/PT0001b/2020,20B,"C241T,C3037T,C14408T,A23403G,C27046T,G28881A,G...",NaN,NaN
4,Portugal/PT0003/2020,20A,"C241T,C3037T,C14408T,A23403G,C29144T",NaN,NaN
5,Portugal/PT0004/2020,20A,"C241T,C3037T,C14408T,A23403G,C29144T",NaN,NaN
6,Portugal/PT0005/2020,20A,"C241T,C335T,C3037T,C14408T,G20398A,A23403G,A26...",NaN,NaN
7,Portugal/PT0006a/2020,20A,"C241T,C3037T,C14408T,A23403G,C29144T",NaN,NaN
8,Portugal/PT0006b/2020,20A,"C241T,C3037T,C14408T,A23403G,C28310T,C29144T,T...",NaN,NaN
9,Portugal/PT0007/2020,20B,"C241T,C3037T,C3373A,C14408T,A23403G,G28881A,G2...",NaN,NaN


In [54]:
lst_separated = []
populated_indices = []

for line_del in final_df['deletions']: #fazer o split das , e -

	if type(line_del) != str and math.isnan(line_del):
		lst_separated.append(line_del)
		continue
	else:
		pos = re.split('-|,', line_del)
		lst_separated.append(pos)

for element,pos_indices in zip(lst_separated, range(0,len(pca_df))): #se o tamanho for impar, remover o ultimo elemento da lista - so vamos ver as deleçoes que tem posiçao1:posiçao2
	if type(element) != list and math.isnan(element):
		continue
	elif len(element) %2 != 0: 
		element.pop()
		
	populated_indices.append(int(pos_indices))


# print(lst_separated[:150])

# print(populated_indices[:150])



In [153]:

positions = []

for i in range(0,len(lst_separated)+1):
	if lst_separated[populated_indices[i]] == []:
		positions.append(populated_indices[i])
		var1 = int(populated_indices[i])
		#print(populated_indices[i],var1, lst_separated[var1])
		lst_separated.pop(var1)
	else:
		continue

IndexError: list index out of range

In [88]:
#TODO: tentar colocar os pares em tuples assim - [[(1602,5010),(1242,12412)...], [(1223,1232)]]
outcome_del = []

for index in range(0,len(pca_df)):
	i = 0
	j = 1
	k = 0
	if lst_separated[populated_indices[index]] == []:
		k += 1
		# positions.append(populated_indices[i])
		# var1 = int(populated_indices[i])
		# #print(populated_indices[i],var1, lst_separated[var1])
		# lst_separated.pop(var1)
		continue
	else:
		while k < len(lst_separated[populated_indices[index]])/2:
			pos1 = int(lst_separated[populated_indices[index]][i])
			pos2 = int(lst_separated[populated_indices[index]][j])
			i += 2
			j += 2
			k += 1
			#print(pos1,pos2)
			var = str(record_dict_2["Wuhan/Hu-1/2019"].seq)[pos1-1:pos2-1]
			var = var + ">del"
			outcome_del.append([populated_indices[index], var])



	#print(lst_separated[subindex])



IndexError: list index out of range

## Inserções

In [ ]:
#Em relaçao a inserçao, 0 based (-1 em cada posiçao no slicing)
record_dict = SeqIO.to_dict(SeqIO.parse("sequences.fasta", "fasta"))
print(str(record_dict["Portugal/PT3927/2021"].seq)[11288:11296])

In [ ]:
print(final_df['insertions'][1])
#type(nextclade_2['insertions'][1])

if  math.isnan(final_df['insertions'][1]):
	print("NaN")
	continue
else:
	print("Not NaN")

In [81]:
final_df['insertions']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
20105    NaN
20106    NaN
20107    NaN
20108    NaN
20109    NaN
Name: insertions, Length: 20110, dtype: object

In [78]:
lst_separated_ins = []
populated_indices_ins = []

for line_del in final_df['insertions']: #fazer o split das , e -

	if type(line_del) != str and math.isnan(line_del):
		lst_separated.append(line_del)
		continue
	else:
		pos = re.split('-|,', line_del)
		lst_separated_ins.append(pos)

for element,pos_indices in zip(lst_separated_ins, range(0,len(pca_df))): #se o tamanho for impar, remover o ultimo elemento da lista - so vamos ver as deleçoes que tem posiçao1:posiçao2
	if type(element) != list and math.isnan(element):
		continue
	elif len(element) %2 != 0: 
		element.pop()
		
	populated_indices_ins.append(int(pos_indices))


print(lst_separated_ins[:150])

# print(populated_indices[:150])


[[], [], [], [], [], [], [], [], [], [], ['27381:C', '27386:CTC'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['28251:CTG', '28266:AACA'], [], [], [], [], [], [], [], [], [], [], [], []]


In [ ]:
#TODO: tentar colocar os pares em tuples assim - [[(1602,5010),(1242,12412)...], [(1223,1232)]]
outcome_ins = []

for seqName,index in zip(pca_df['seqName'], range(0,len(pca_df))):
	i = 0
	j = 1
	k = 0

	if lst_separated[populated_indices[index]] == []:
		k += 1
		# positions.append(populated_indices[i])
		# var1 = int(populated_indices[i])
		# #print(populated_indices[i],var1, lst_separated[var1])
		# lst_separated.pop(var1)
		continue
	else:
		while k < len(lst_separated[populated_indices[index]])/2:
			pos1 = int(lst_separated[populated_indices[index]][i])
			pos2 = int(lst_separated[populated_indices[index]][j])
			i += 2
			j += 2
			k += 1
			#print(pos1,pos2)
			var = str(record_dict_2[seqName].seq)[pos1-1:pos2-1]
			var = var + ">del"
			outcome_in.append([populated_indices[index], var])



	#print(lst_separated[subindex])
